In [1]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)

import librosa
import matplotlib.pyplot as plt
!pip install librosa
!pip install transformers torch torchaudio

2.8.0+cu126
2.8.0+cu126


In [2]:
from IPython.display import Audio
from matplotlib.patches import Rectangle
from torchaudio.utils import download_asset

torch.random.manual_seed(0)

In [3]:
# 파일 경로를 로컬 m4a 파일로 변경
M4A_FILE_PATH = "/content/911.m4a"

try:
    SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(M4A_FILE_PATH)

except Exception as e:
    print(f"m4a 파일 로드 실패: {e}")

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [4]:
#sample_rate 16000으로 변경
target_sample_rate = 16000
if SAMPLE_RATE != target_sample_rate:
    resampler = torchaudio.transforms.Resample(orig_freq=SAMPLE_RATE, new_freq=target_sample_rate)
    resampled_waveform = resampler(SPEECH_WAVEFORM)
else:
    resampled_waveform = SPEECH_WAVEFORM

In [5]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
input_values = processor(resampled_waveform.squeeze(), return_tensors="pt", sampling_rate=target_sample_rate).input_values

In [7]:
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)

In [8]:
print("-" * 20)
print(f"인식된 문장: {transcription[0]}")
print("-" * 20)

--------------------
인식된 문장: NINE ONE ONE WHAT IS YOUR EMERGENCY
--------------------
